In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
pd.options.mode.chained_assignment = None

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import pandas as pd

# Update the path below based on where the file is located in your Google Drive
file_path = '/content/drive/My Drive/amharic.xlsx'

# Load the CSV file
df = pd.read_excel(file_path)

# Display the first few rows of the DataFrame



Mounted at /content/drive


In [ ]:
# Display the first few rows of the DataFrame
df = pd.read_excel(file_path)
print(df.head(5))

                                            Comments Sentiment Unnamed: 2  \
0  እባካችሁ በትግራይ ክልል 2ኛ እና 3ኛ ዲግሪ ሲማሩ ለቆዩ ተማሪዎች ጉዳይ...         0        NaN   
1                          ነጭ ሩዝ በምስር? ለፍየል የማይሰጥ ፋፋ         2        NaN   
2          አንተ ልዩ ሰዉ ነህ እስከ ህይወት መስዋዕትነት ድረስ ከጎንህ ነን         2        NaN   
3                               አንተ ደደብ የሆንክ አፍህን ዝጋ         0        NaN   
4  ትምህርት ሚኒስቴር እየሰራ ያለው ሥራ ምንም አልገባኝ አለ በየዓመቱ ኮርስ...         0        NaN   

   Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7  Unnamed: 8  
0         NaN         NaN         NaN         NaN         NaN         NaN  
1         NaN         NaN         NaN         NaN         NaN         NaN  
2         NaN         NaN         NaN         NaN         NaN         NaN  
3         NaN         NaN         NaN         NaN         NaN         NaN  
4         NaN         NaN         NaN         NaN         NaN         NaN  


In [ ]:
# Check if each row in the Sentiment column is a string
string_rows = df['Sentiment'].apply(lambda x: isinstance(x, str))
# Filter out rows where Sentiment is a string
cleaned_data = df[~string_rows]
df['Sentiment'] = pd.to_numeric(df['Sentiment'], errors='coerce')

# Optional: Handle NaN values. Here's an example of dropping them
df.dropna(subset=['Sentiment'], inplace=True)

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Remove rows where the 'sentiment' column is 2 or 3
df = df[~df['Sentiment'].isin([2, 3])]

# Now df will only contain rows where the sentiment is not equal to 2 or 3

In [ ]:
df = df[['Comments', 'Sentiment']]
df['sentiments'] = df['Sentiment'].apply(lambda x: 'positive' if x >= 1 else 'negative')

df = df[['Comments', 'sentiments']]
df = df.sample(frac=1).reset_index(drop=True)
sentiment_counts = df['sentiments'].value_counts()
print(sentiment_counts)

sentiments
negative    3553
positive    1788
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame containing the comments and sentiments
# Clean and preprocess the data
df['Comments'] = df['Comments'].fillna('')  # Replace NaN with empty string
df['Comments'] = df['Comments'].astype(str)  # Ensure all data is string type

# Features and target variable
X = df['Comments']
y = df['sentiments']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataFrame for training and testing
df_train = pd.DataFrame({'Comments': X_train, 'sentiments': y_train})
df_test = pd.DataFrame({'Comments': X_test, 'sentiments': y_test})



In [ ]:
import re

def clean_text(row, options):
    """Removes url, mentions, emoji, and converts to lowercase if applicable"""
    # Ensure row is a string
    if not isinstance(row, str):
        # If row is not a string, return it as is or convert it to a string as per your requirement
        return row  # Or, return str(row) if you want to convert non-strings to strings

    if options['lowercase']:
        row = row.lower()

    if options['remove_url']:
        row = re.sub(r"(?:\@|https?\://)\S+", "", row)

    if options['remove_mentions']:
        row = re.sub("@[A-Za-z0-9_]+","", row)

    if options['demojify']:
        emoj = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            u"\U00002702-\U000027B0"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642"
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  # dingbats
            u"\u3030"
                          "]+", re.UNICODE)
        row = re.sub(emoj, '', row)

    return row
clean_config = {
    'remove_url': True,
    'remove_mentions': True,
    'lowercase': True,
    'demojify': True
    }

In [ ]:

df_test['Comments'] = df_test['Comments'].apply(clean_text, args=(clean_config,))
df_train['sentiments'] = df_train['sentiments'].apply(clean_text, args=(clean_config,))

In [ ]:
pip install nlpaug


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.7 MB/s eta 0:00:00


In [ ]:
#character level normalization

import re
#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalize_char_level_missmatch(input_token):
  # Check if the input is not a string
    if not isinstance(input_token, str):
        # Return the input as is or convert it to a string, depending on your needs
        return input_token  # or return str(input_token) to convert to string
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    #Normalizing words with Labialized Amharic characters such as በልቱዋል or  በልቱአል to  በልቷል
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48



In [ ]:
df_test['Comments'] = df_test['Comments'].apply(lambda x: normalize_char_level_missmatch(x))
df_train['Comments'] = df_train['Comments'].apply(lambda x: normalize_char_level_missmatch(x))

In [ ]:
import nlpaug.augmenter.word as naw
import pandas as pd
from sklearn.model_selection import train_test_split

# Function to augment data
def augment_text(data, augmenter, repetitions=1):
    augmented_texts = []
    for text in data:
        # Ensure text is a string
        text = str(text)
        for _ in range(repetitions):
            augmented = augmenter.augment(text)
            augmented_texts.append(str(augmented))  # Ensure the augmented text is a string
    return augmented_texts

# Assuming df is already loaded and contains the columns 'Review' and 'Liked'
# Split your data
#X_train, X_test, y_train, y_test = train_test_split(df['Review'], df['Liked'], test_size=0.2, random_state=42)

# Initialize augmenter - here using synonym replacement
augmenter = naw.SynonymAug(aug_src='wordnet')

# Augment data - now tripling the data
augmented_texts = augment_text(X_train, augmenter, repetitions=3)  # Tripling data with augmentation

# Reset index for y_train to properly repeat it
y_train_reset = y_train.reset_index(drop=True)

# Extend the original training data with augmented data
augmented_data = pd.DataFrame({'Comments': augmented_texts, 'sentiments': y_train_reset.repeat(3).values})  # Repeat y_train three times
X_train_augmented = pd.concat([X_train, augmented_data['Comments']]).reset_index(drop=True)
y_train_augmented = pd.concat([y_train, augmented_data['sentiments']]).reset_index(drop=True)

# Verification that all entries are strings
if any(not isinstance(x, str) for x in X_train_augmented):
    print("Non-string elements detected in augmented data.")

assert all(isinstance(x, str) for x in X_train_augmented), "All elements in X_train_augmented must be strings."

# Now you have a significantly augmented dataset
print("Original training data size:", len(X_train))
print("Augmented training data size:", len(X_train_augmented))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Original training data size: 4272
Augmented training data size: 17088


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_pipeline_imblearn

class StackingSentiment:
    def __init__(self):
        # Define base models with their respective pipelines
        base_models = [
            ('naive_bayes', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5),
                SMOTE(),
                MultinomialNB(alpha=0.1)
            )),
            ('svm', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5),
                SMOTE(),
                SVC(probability=True, kernel='linear', C=1)
            )),
            ('random_forest', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5),
                SMOTE(),
                RandomForestClassifier(n_estimators=100)
            )),
            ('log_reg', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5),
                SMOTE(),
                LogisticRegression(C=1)
            ))
        ]

        # Define the final estimator
        final_estimator = LogisticRegression()

        # Define the stacking classifier with cross-validation
        self.pipeline = StackingClassifier(
            estimators=base_models,
            final_estimator=final_estimator,
            cv=5
        )

    def fit(self, X_train, y_train):
        self.pipeline.fit(X_train, y_train)

    def predict(self, X):
        return self.pipeline.predict(X)

    def predict_proba(self, X):
        return self.pipeline.predict_proba(X)

# Example usage
if __name__ == "__main__":
    # Assume X_train, X_test, y_train, y_test are predefined

    stacking_sentiment = StackingSentiment()
    stacking_sentiment.fit(X_train, y_train)  # Train the model
    predictions = stacking_sentiment.predict(X_test)  # Predict test data
    accuracy = accuracy_score(y_test, predictions)  # Calculate accuracy
    print(f"Accuracy of the stacking classifier: {accuracy:.2f}")


In [ ]:
pip install scikit-learn imbalanced-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as make_pipeline_imblearn

# Define Amharic stop words list
amharic_stop_words = [
    "ህ-ን", "እንደ", "የ", "አል", "ው", "ኡ", "በ", "ተ", "ለ", "ን", "ኦች", "ኧ", "ና", "ከ", "አቸው", "ት", "መ",
    "አ", "አት", "ዎች", "ም", "አስ", "ኡት", "ላ", "ይ", "ማ", "ያ", "አ", "ቶ", "እንዲ", "የሚ", "ኦ", "ይ", "እየ",
    "ሲ", "ብ", "ወደ", "ሌላ", "ጋር", "ኡ", "እዚህ", "አንድ", "ውስጥ", "እንድ", "እ-ል", "ን-ብ-ር", "በኩል", "ል",
    "እስከ", "እና", "ድ-ግ-ም", "መካከል", "ኧት", "ሊ", "አይ", "ምክንያት", "ይህ", "ኧች", "ኢት", "ዋና", "አን",
    "እየ", "ስለ", "ች", "ስ", "ቢ", "ብቻ", "በየ", "ባለ", "ጋራ", "ኋላ", "እነ", "አም", "ሽ", "አዊ", "ዋ", "ያለ",
    "ግን", "ምን", "አችን", "ወይዘሮ", "ወዲህ", "ማን", "ዘንድ", "የት", "ናቸው", "ላ", "ይሁን", "ወይም", "ታች",
    "እዚያ", "እጅግ", "እንጅ", "በጣም", "ወዘተ", "ጅ-ም-ር", "አሁን", "ከነ", "ተራ", "ም-ል", "ጎሽ", "አዎ", "እሽ",
    "ጉዳይ", "ረገድ", "ያህል", "ይልቅ", "ዳር", "እንኳ", "አዎን", "ብ-ዝ", "ጥቂት", "እኔ", "አንተ", "እርስዎ",
    "እሳቸው", "እሱ", "አንች", "እኛ", "እነሱ", "እናንተ", "ይኸ", "የቱ", "መቼ", "ወዲያ", "ወዴት", "እንዴት",
    "ልክ", "አጠገብ", "ባሻገር", "እንትን", "እንትና", "ሁሉ", "እንጂ", "ይች", "ናት", "ምናልባት", "በቀር", "እስኪ",
    "ወይ", "እንዴ", "ስንት", "መቸ", "ከፍ", "ቢያንስ", "ብ-ቅ", "ምሳሌ", "እንግዲ", "እሷ", "ምነው", "የተለያዩ",
    "ወይስ", "እርስወት", "እንቶኔ", "እንቶኒት", "ኢ", "ኛ", "ነት", "በት", "ኤት", "ኤ", "ለይ", "ኦት", "ህ-ድ", "ዊ",
    "እን", "ኧች-ች", "ሚ", "ኦች-ች", "ም-ት", "እዳ", "በየት", "ኤን", "ል-ይ", "ኢን", "ም-ን", "ስት"
]

from imblearn.pipeline import make_pipeline as make_pipeline_imblearn

class AdvancedStackingSentiment:
    def __init__(self):
        # Define base models with their respective pipelines
        base_models = [
            ('naive_bayes', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5, stop_words=amharic_stop_words),
                SMOTE(),
                MultinomialNB(alpha=0.1)
            )),
            ('svm', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5, stop_words=amharic_stop_words),
                SMOTE(),
                SVC(probability=True, kernel='linear', C=1)
            )),
            ('random_forest', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5, stop_words=amharic_stop_words),
                SMOTE(),
                RandomForestClassifier(n_estimators=100)
            )),
            ('log_reg', make_pipeline_imblearn(
                TfidfVectorizer(ngram_range=(1, 7), analyzer='char', max_df=0.85, min_df=5, stop_words=amharic_stop_words),
                SMOTE(),
                LogisticRegression(C=1)
            ))
        ]

        # Define the final estimator
        final_estimator = LogisticRegression()

        # Define the stacking classifier with cross-validation
        self.pipeline = StackingClassifier(
            estimators=base_models,
            final_estimator=final_estimator,
            cv=5
        )

    def fit(self, X_train, y_train):
        self.pipeline.fit(X_train, y_train)

    def predict(self, X):
        return self.pipeline.predict(X)

    def predict_proba(self, X):
        return self.pipeline.predict_proba(X)



# Example usage
if __name__ == "__main__":
    # Example data loading
    # X_train, X_test, y_train, y_test should be initialized here
    stacking_sentiment = AdvancedStackingSentiment()
    stacking_sentiment.fit(X_train, y_train)
    predictions = stacking_sentiment.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy of the stacking classifier: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: Th

Accuracy of the stacking classifier: 0.86
